In [4]:
import kagglehub
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler

kagglehub.login()

# Gradient Boost

This file turns the Gradient Boost Model into a .pkl file so we can use it in our frontend.

## Import and Inspect the Data

Import the data set.

In [5]:
dataset_path = kagglehub.dataset_download("maharshipandya/-spotify-tracks-dataset")
print(dataset_path)

100%|██████████| 8.17M/8.17M [00:00<00:00, 13.7MB/s]

Extracting files...


/Users/ainekeenan/.cache/kagglehub/datasets/maharshipandya/-spotify-tracks-dataset/versions/1


Check how the data was imported. It is 1 CSV.

Get the mean, count, and percentiles for each feature.

In [8]:
from sklearn.preprocessing import OneHotEncoder

df = pd.read_csv(dataset_path + "/dataset.csv")

onehot = OneHotEncoder(sparse_output=False)
genre_encoded = onehot.fit_transform(df[['track_genre']])
genre_names = onehot.categories_[0]

genre_encoded_df = pd.DataFrame(genre_encoded, columns=genre_names)

columns_to_drop = ['track_id', 'artists', 'album_name', 'track_name', 'track_genre']
df_with_encoded_genres = pd.concat([df, genre_encoded_df], axis=1)
df_with_encoded_genres = df_with_encoded_genres.drop(columns=columns_to_drop)

print("Shape of final dataframe with encoded genres:", df_with_encoded_genres.shape)
print("\nColumns in the final dataframe:")
print(df_with_encoded_genres.columns.tolist())

print("\nFirst few rows of the cleaned dataframe (including track_genre):")
print(df_with_encoded_genres.head())

Shape of final dataframe with encoded genres: (114000, 130)

Columns in the final dataframe:
['Unnamed: 0', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-p

In [9]:
mmscaler = MinMaxScaler()
mm_datax = mmscaler.fit_transform(df_with_encoded_genres)
df_mm_data = pd.DataFrame(mm_datax, columns=df_with_encoded_genres.columns)
print(df_mm_data.columns.tolist())
df_mm_data.head()

['Unnamed: 0', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'met

,Unnamed: 0,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,...,spanish,study,swedish,synth-pop,tango,techno,trance,trip-hop,turkish,world-music
0,0.000000,0.73,0.044043,0.0,0.686294,0.4610,0.090909,0.791392,0.0,0.148187,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000009,0.55,0.028566,0.0,0.426396,0.1660,0.090909,0.597377,1.0,0.079067,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000018,0.57,0.040255,0.0,0.444670,0.3590,0.000000,0.736123,1.0,0.057720,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000026,0.71,0.038557,0.0,0.270051,0.0596,0.000000,0.573701,1.0,0.037617,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000035,0.82,0.037969,0.0,0.627411,0.4430,0.181818,0.737103,1.0,0.054508,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#Gradient Boost

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
import joblib

numerical_columns = ['popularity','duration_ms', 'danceability', 'energy', 'key',
                    'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
                    'liveness', 'valence', 'tempo', 'time_signature']

genre_columns = df_with_encoded_genres.columns.difference(numerical_columns)
feature_columns = [col for col in numerical_columns if col != 'popularity']
feature_columns.extend(genre_columns)

feature_columns = [col for col in feature_columns if not col.startswith('Unnamed')]

X = df_mm_data[feature_columns]
y = df_mm_data['popularity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=24)

xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nModel Performance Metrics:")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"R-squared Score: {r2:.4f}")

joblib.dump(xgb_model, 'models/gradient_boost.pkl')
joblib.dump(feature_columns, 'models/feature_columns_gb.pkl')



Model Performance Metrics:
Root Mean Squared Error: 0.1909
Mean Absolute Error: 0.1485
R-squared Score: 0.2711


['models/feature_columns_gb.pkl']